In [2]:
import os.path
import pickle
import pandas
import numpy
import gzip

mainPath='/home/frubio/AVA/'

In [2]:
challenge_dict={}
f = open(mainPath+'AVA_dataset/challenges.txt', 'r')
for line in f:
    strSplit=line.split()
    challenge_dict[int(strSplit[0])]=strSplit[1]

f.close()

In [3]:
tags_dict={}
f = open(mainPath+'AVA_dataset/tags.txt', 'r')
for line in f:
    strSplit=line.split()
    tags_dict[int(strSplit[0])]=strSplit[1]

f.close()

In [4]:
styles_dict={}
f = open(mainPath+'AVA_dataset/style_image_lists/styles.txt', 'r')
for line in f:
    strSplit=line.split()
    styles_dict[int(strSplit[0])]=strSplit[1]

f.close()

In [5]:
trainStyle_dict={}
f1 = open(mainPath+'AVA_dataset/style_image_lists/train.jpgl', 'r')
f2 = open(mainPath+'AVA_dataset/style_image_lists/train.lab', 'r')

for line in f1:
    auxLine=f2.readline()
    trainStyle_dict[int(line)]=int(auxLine)

f1.close()
f2.close()

In [6]:
testStyle_dict={}
f1 = open(mainPath+'AVA_dataset/style_image_lists/test.jpgl', 'r')
f2 = open(mainPath+'AVA_dataset/style_image_lists/test.multilab', 'r')

for line in f1:
    auxLine=f2.readline()
    testStyle_dict[int(line)]=map(int, auxLine.split())

f1.close()
f2.close()

In [7]:
aesthetics_dict={1:'animal',2:'architecture',3:'cityscape',4:'floral',
                 5:'fooddrink',6:'landscape',7:'portrait',8:'stilllife'}

ttAesthetics_dict={}

count=1
for i in aesthetics_dict:
    f1 = open(mainPath+'AVA_dataset/aesthetics_image_lists/'+aesthetics_dict[i]+'_train.jpgl', 'r')
    for line in f1:
        if int(line) in ttAesthetics_dict:
            print line
            aux=ttAesthetics_dict[int(line)]
            if isinstance(aux,list):
                ttAesthetics_dict[int(line)].append(count)
            else:
                ttAesthetics_dict[int(line)]=[ttAesthetics_dict[int(line)],count]
        else:
            ttAesthetics_dict[int(line)]=count
    f1.close()
    
    f1 = open(mainPath+'AVA_dataset/aesthetics_image_lists/'+aesthetics_dict[i]+'_test.jpgl', 'r')
    for line in f1:
        if int(line) in ttAesthetics_dict:
            print line
            aux=ttAesthetics_dict[int(line)]
            if isinstance(aux,list):
                ttAesthetics_dict[int(line)].append(count)
            else:
                ttAesthetics_dict[int(line)]=[ttAesthetics_dict[int(line)],count]
        else:
            ttAesthetics_dict[int(line)]=count
    f1.close()
    
    count+=1


In [8]:
vector=[]
f = open(mainPath+'AVA_dataset/AVA.txt', 'r')

for line in f:
    strSplit=line.split()
    fname=mainPath+'images/'+strSplit[1]+'.jpg'
    if os.path.isfile(fname):
        vector.append(map(int, strSplit))
        
f.close()
        
npArray=numpy.array(vector)
votes=['vote'+str(s) for s in range(1,11)]
myList=['line','id']+votes+['tag1','tag2','challenge']

data=pandas.DataFrame(npArray,columns=myList)

In [9]:
for i in styles_dict:
    data.loc[:,styles_dict[i]] = numpy.nan

In [10]:
for i in aesthetics_dict:
    data.loc[:,aesthetics_dict[i]] = numpy.nan

In [11]:
for k in range(0,len(data)):
    if int(data.iloc[k,1]) in trainStyle_dict:
        data.iloc[k,14+trainStyle_dict[data.iloc[k,1]]]=1

    if int(data.iloc[k,1]) in testStyle_dict:
        count=15
        for i in testStyle_dict[data.iloc[k,1]]:
            if i==1:
                data.iloc[k,count]=1
            else:
                data.iloc[k,count]=0
            count+=1
    
    if int(data.iloc[k,1]) in ttAesthetics_dict:
        valueAES=ttAesthetics_dict[data.iloc[k,1]]
        if isinstance(valueAES,list):
            for i in valueAES:
                data.iloc[k,28+i]=1
        else:
            data.iloc[k,28+valueAES]=1
    

In [22]:
pickle.dump( data, gzip.open( "info.pklz", "wb" ),2)


In [4]:
pickle.dump( tags_dict, gzip.open( "tags.pklz", "wb" ),2)